In [1]:

import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

!pip install pytubefix


# Clean up the notebook
clean_notebook()

Notebook cleaned.


In [2]:
from pytubefix import YouTube
import subprocess
import os

video_url = "https://youtu.be/C516g4gmAuQ?si=6c0PeWFeBv2m30LP"

# Time range (format: HH:MM:SS or seconds)
start_time = "00:03:47"  # Start at 1 minute 30 seconds
end_time = "00:05:56"    # End at 3 minutes

yt = YouTube(video_url)
print(f"Title: {yt.title}")

# Download full video
video_stream = yt.streams.filter(
    file_extension='mp4',
    only_video=True
).order_by('resolution').desc().first()

temp_file = "temp_full_video.mp4"
output_file = "muay_thai_clip.mp4"

print(f"Downloading: {video_stream.resolution}")
video_stream.download(filename=temp_file)

# Extract segment and encode to H.264
print(f"Extracting segment: {start_time} to {end_time}")
subprocess.run([
    "ffmpeg", "-y",
    "-i", temp_file,
    "-ss", start_time,      # Start time
    "-to", end_time,        # End time
    "-c:v", "libx264",      # H.264 video codec
    "-preset", "medium",    # Encoding speed (ultrafast, fast, medium, slow, veryslow)
    "-crf", "23",           # Quality (0-51, lower = better quality)
    "-c:a", "aac",          # AAC audio codec
    "-b:a", "128k",         # Audio bitrate
    "-movflags", "+faststart",  # Optimize for web streaming
    output_file
], check=True)

# Clean up temp file
os.remove(temp_file)
print(f"Done! Saved as {output_file}")

Title: ตัวอย่างไหว้ครูมวยไทย และแม่ไม้มวยไทย
Downloading: 1080p
Extracting segment: 00:03:47 to 00:05:56


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Done! Saved as muay_thai_clip.mp4


[out#0/mp4 @ 0x5a3e574083c0] video:28684kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.158313%
frame= 3866 fps=218 q=-1.0 Lsize=   28730kB time=00:02:08.92 bitrate=1825.5kbits/s speed=7.28x    
[libx264 @ 0x5a3e57446380] frame I:16    Avg QP:13.69  size:116588
[libx264 @ 0x5a3e57446380] frame P:1140  Avg QP:18.28  size: 16976
[libx264 @ 0x5a3e57446380] frame B:2710  Avg QP:23.07  size:  3009
[libx264 @ 0x5a3e57446380] consecutive B-frames:  4.9%  3.3%  4.6% 87.2%
[libx264 @ 0x5a3e57446380] mb I  I16..4: 18.6% 65.8% 15.6%
[libx264 @ 0x5a3e57446380] mb P  I16..4:  0.7%  2.5%  0.4%  P16..4: 13.5%  5.7%  2.9%  0.0%  0.0%    skip:74.3%
[libx264 @ 0x5a3e57446380] mb B  I16..4:  0.0%  0.1%  0.0%  B16..8: 12.2%  1.0%  0.2%  direct: 0.5%  skip:85.9%  L0:41.4% L1:48.3% BI:10.3%
[libx264 @ 0x5a3e57446380] 8x8 transform intra:68.0% inter:70.3%
[libx264 @ 0x5a3e57446380] coded y,uvDC,uvAC intra: 48.0% 38.5% 15.6% inter: 4.1% 2.9% 0.1%
[libx264 @ 0x5a3e57446380] i1

# Display Video

In [3]:
import cv2
from IPython.display import display, Image, clear_output
import time

def display_video(video_path: str, width: int = 640, speed: float = 1.0, skip_frames: int = 0):
    """
    Display a video file in Jupyter Notebook with detailed progress bar.
    
    Parameters:
        video_path: Path to the video file
        width: Target width for display (maintains aspect ratio)
        speed: Playback speed multiplier
        skip_frames: Number of frames to skip between each displayed frame
                     (0 = show all frames, 1 = show every 2nd frame, 2 = show every 3rd frame, etc.)
    """
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Cannot open video file: {video_path}")
        return
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if fps <= 0:
        fps = 30
    
    total_seconds = total_frames / fps
    delay = 1.0 / (fps * speed)
    
    # Progress bar settings
    bar_height = 20
    bar_color = (0, 200, 0)        # Green
    bar_bg_color = (40, 40, 40)    # Dark gray
    text_color = (255, 255, 255)   # White
    
    try:
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            
            if not ret:
                break
            
            frame_count += 1
            
            # Skip frames if needed
            if skip_frames > 0 and (frame_count - 1) % (skip_frames + 1) != 0:
                continue
            
            # Resize frame maintaining aspect ratio
            h, w = frame.shape[:2]
            aspect_ratio = h / w
            target_height = int(width * aspect_ratio)
            frame = cv2.resize(frame, (width, target_height))
            
            # Calculate time
            current_seconds = frame_count / fps
            current_time = f"{int(current_seconds // 60):02d}:{int(current_seconds % 60):02d}"
            total_time = f"{int(total_seconds // 60):02d}:{int(total_seconds % 60):02d}"
            
            # Draw progress bar background
            cv2.rectangle(frame, 
                          (0, target_height - bar_height), 
                          (width, target_height), 
                          bar_bg_color, -1)
            
            # Draw progress bar
            progress = frame_count / total_frames
            progress_width = int(width * progress)
            cv2.rectangle(frame, 
                          (0, target_height - bar_height), 
                          (progress_width, target_height), 
                          bar_color, -1)
            
            # Draw time text
            time_text = f"{current_time} / {total_time}"
            cv2.putText(frame, time_text, 
                        (10, target_height - 5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        0.5, text_color, 1)
            
            # Draw speed and skip info
            info_text = ""
            if speed != 1.0:
                info_text += f"{speed}x"
            if skip_frames > 0:
                if info_text:
                    info_text += " | "
                info_text += f"skip:{skip_frames}"
            
            if info_text:
                text_size = cv2.getTextSize(info_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
                cv2.putText(frame, info_text, 
                            (width - text_size[0] - 10, target_height - 5), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            0.5, text_color, 1)
            
            # Convert to JPEG and display
            _, buffer = cv2.imencode('.jpg', frame)
            display(Image(data=buffer.tobytes()))
            clear_output(wait=True)
            
            time.sleep(delay)
                
    except KeyboardInterrupt:
        print("Playback interrupted.")
    finally:
        cap.release()
        print("Video stream ended.")

In [4]:
display_video("./muay_thai_clip.mp4", width=1024, speed=1.0, skip_frames=20)

Video stream ended.
